In [62]:
import pandas as pd
import numpy
import random as rd
from tqdm import tqdm_notebook
import os
import torch
from torch.utils import data
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# 장혁이형 테스트할 때 순서대로 쭉 실행해보세요오~

In [54]:
"""
TrainDataset

- msmarco-docs.tsv 데이터는 크기가 너무 크고, train, test에 모두 쓰이므로 
  각 클래스 변수로 두지 않고 global로 두는 게 나을 듯 ?

    __init__(self, root)
        - self.qid_to_query (dict) : qeury 정보, qid를 key로, qeury를 value로 가짐
        - self.rel_qid_to_docid (dict) : qrels 정보, qid를 key로 해당 q에 대해 관련 docid를 value로 가짐
        - self.top100 (pd.DataFrame) : bing에 의해 뽑힌 query 관련 100개 문서 
        
    __len__(self)
        - qrels(rel_qid_to_docid) 길이 * 3 
        - 1개의 query 당 qrels에서 1개, qrels 아닌 곳(일단은 top100)에서 2개
        
    __getitem__(self, index)
        - q 길이 제한 20 token
        - d 길이 제한 489 token (512 - 3 - 20)
        - tokenizing 전에 너무 긴 (token 제한 * 10 보다 긴) 것들 사전 truncate
        - index를 3으로 나눈 나머지 0 => 연관 있는 문서 from qrels
        - index를 3으로 나눈 나머지 1,2 => 연관 없는 문서 from top100
        - output: q_token_id(int list), d_token_id(int list), rel(int 0 or 1)
        
"""
class TrainDataset(data.Dataset):

    def __init__(self, root=''):
        queries = pd.read_csv(os.path.join(root, 'msmarco-doctrain-queries.tsv'), sep='\t', header=None)
        queries.columns = ['qid', 'query']
        self.qid_to_query = dict(zip(queries['qid'], queries['query']))
        
        qrels = pd.read_csv(os.path.join(root, 'msmarco-doctrain-qrels.tsv'), sep=' ', header=None)
        qrels.columns = ['qid', '0', 'docid', '1']
        tmp_qid_list = list(qrels['qid'])
        self.qid_list = []
        for qid in tmp_qid_list:
            self.qid_list.extend([qid, qid, qid])
        self.rel_qid_to_docid = dict(zip(qrels['qid'], qrels['docid']))
        
        # msmarco-doctrain-top100 읽는 데 너무 오래 걸리면 global로 쓰자
        self.top100 = pd.read_csv('msmarco-doctrain-top100', sep=' ', header=None)
        self.top100.columns = ['qid', 'Q0', 'docid', 'rank', '-1', '-1']

    def __len__(self):
        return len(self.qid_list)

    def __getitem__(self, index):
        'Generates one sample of data'
        qid = self.qid_list[index]
        rel_docid = rel_qid_to_docid[qid]

        # 2/3 는 qrels에 없는(y == 0) 데이터
        if index % 3:
            y = 0
            docid_list = list(self.top100[self.top100['qid'] == qid]['docid'])
            docid_list.remove(rel_docid)
            docid = rd.choice(docid_list)
        # 1/3 는 qrels에 있는(y == 0) 데이터 
        else:
            y = 1
            docid = rel_docid
            
        # Tokenizing query and doc
        q = self.qid_to_query[qid]
        d = docid_to_doc[docid]
        
        # q, d 길이 제한 선행 처리 (너무 긴 경우)
        # (평균 token_len : text_len = 0.22 : 1)
        
        # q text len이 20(q 토큰 제한) * 10보다 큰 경우 truncate
        if len(q) > 200:
            q = q[:200]
        # d text len이 500(q 토큰 제한) * 10보다 큰 경우 truncate
        if len(d) > 5000:
            d = d[:5000]
        
        # tokenize
        q_token = tokenizer.tokenize(q)
        q_token_len = len(q_token)
        if q_token_len > 20:
            q_token_len = 20
            q_token = q_token[:q_token_len]
        else:
            q_token.extend(['[PAD]'] * (20 - q_token_len))
        q_token_id = tokenizer.convert_tokens_to_ids(q_token)
        
        d_token = tokenizer.tokenize(d)
        d_token_len = len(d_token)
        if q_token_len > 489: # 512 - 3 ([CLS], [SEP], [SEP])
            d_token_len = 489
            d_token = d_token[:d_token_len]
        else:
            q_token.extend(['[PAD]'] * (489 - d_token_len))
        d_token_id = tokenizer.convert_tokens_to_ids(d_token)
        
        assert len(q_token_id) == 20
        assert len(d_token_id) == 489
        
        return q_token_id, d_token_id, y

In [ ]:
# {file_path} 이 부분에 파일 위치 적기
docs = pd.read_csv(os.path.join({file_path} + 'msmarco-docs.tsv'), sep='\t', header=None)
docs.columns = ['docid', 'url', 'title', 'body']
docid_to_doc = dict(zip(docs['docid'], docs['body']))

In [ ]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 1}

train_dataset = TrainDataset('')
train_generator = data.DataLoader(train_dataset, **params)

In [ ]:
# data generation 잘 되는지 확인
# 첫번째 배치의 query, doc, labels shape과 내용 출력
# 그 다음부터는 처음 배치랑 shape 다르면 출력

first = True

for local_q, local_d local_labels in train_generator:
    if first:
        local_q_shape = local_q.shape
        local_d_shape = local_d.shape
        local_labels_shape = local_labels.shape
        print(local_q_shape, local_d_shape, local_labels_shape)
        print(local_q, local_d, local_labels)
        print()
        first = False
        
    else:
        if local_q.shape != local_q_shape:
            print('Batch Q Different !!')
            print(local_q.shape)
            print()
        if local_d.shape != local_d_shape:
            print('Batch D Different !!')
            print(local_d.shape)
            print()
        if local_labels.shape != local_labels_shape:
            print('Labels Shape Different !!')
            print(local_labels.shape)
            print()
        

In [44]:
# # make docid_to_doc
# docs_lookup = pd.read_csv("msmarco-docs-lookup.tsv", sep='\t', header=None)
# docs_lookup.columns = ['docid', -1, -1]
# dicid_list = list(docs_lookup['docid'])

# docid_to_doc = dict()
# for docid in tqdm_notebook(docid_list):
#     doc = 'hello ' * rd.randint(100, 6000)
#     docid_to_doc[docid] = doc